In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
from state_cleaner import *
from datetime import datetime
import requests
import json

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
os.chdir('../..')
os.chdir('CovidMobile')
os.getcwd()

'/Users/samismalling/Documents/mobility-report-data-extractor-master/CovidMobile'

In [3]:
fips_to_county = pd.read_csv('./data/ZIP-COUNTY-FIPS_2017-06.csv')
parks = pd.read_csv('./data/parks_only.csv')

In [4]:
fips = parks['FIPS'].unique()
fips = fips[1:]
len(fips)

955

In [5]:
fips_to_zips = dict()
for fip in fips:
    fips_to_zips[int(fip)] = list(fips_to_county[fips_to_county['STCOUNTYFP']==fip]['ZIP'])

In [6]:
def get_weather(locationid, datasetid, begin_date, end_date, mytoken, base_url):
    token = {'token': mytoken}
    
    #passing as string instead of dict because NOAA API does not like percent encoding
    params = 'datasetid='+str(datasetid)+'&'+'locationid='+str(locationid)+'&'+'startdate='+str(begin_date)+'&'+'enddate='+str(end_date)+'&'+'limit=25'+'&datatypeid=TOBS,PRCP'
    
    r = requests.get(base_url+params, headers=token)
    #print("Request status code: "+str(r.status_code))

    try:
        #results comes in json form. Convert to dataframe
        df = pd.DataFrame.from_dict(r.json()['results'])
        #print("Successfully retrieved "+str(len(df['station'].unique()))+" stations")
        dates = pd.to_datetime(df['date'])

        return df
    except:
        pass

In [7]:
def already_exists(fip):
    #Checks to see if the fip data is already in the database
    if conn.noaaweather.records.count_documents({ 'FIPS': fip }, limit = 1) != 0:
        return True
    else:
        return False

In [ ]:
import itertools
z = dict(itertools.islice(fips_to_zips.items(), 5))

In [8]:
import json
import pymongo
import time
from pprint import pprint

conn=pymongo.MongoClient()
db = conn.noaaweather
records = db.records
conn.list_database_names()

['admin', 'config', 'local', 'mydb', 'noaaweather']

In [34]:
baseurl = 'https://www.ncdc.noaa.gov/cdo-web/api/v2/data?'
mytoken = 'NVPWXeGxislWKqThiizoffNjnPtUtAcT'
databaseid = 'GHCND'
begindate = '2020-03-01'
enddate = '2020-05-01'
newdf = get_weather('ZIP:'+str(6673), databaseid,begindate,enddate,mytoken,baseurl)

Request status code: 200


In [ ]:
codes = fips_to_zips[6053]
for code in codes:
    try:
        newdf = get_weather('ZIP:'+str(code), databaseid,begindate,enddate,mytoken,baseurl)
        fip_weather = fip_weather.append(newdf)
    except:
        continue

In [11]:
empty_fips = []

In [ ]:
baseurl = 'https://www.ncdc.noaa.gov/cdo-web/api/v2/data?'
mytoken = 'NVPWXeGxislWKqThiizoffNjnPtUtAcT'
databaseid = 'GHCND'
begindate = '2020-03-01'
enddate = '2020-05-01'

tic = time.perf_counter()
weather = pd.DataFrame()
for fip, codes in fips_to_zips.items():
    if not already_exists(fip) and fip not in empty_fips: #check to make sure not double counting data
        print(fip)
        fip_weather = pd.DataFrame()
        for code in codes:
            try:
                newdf = get_weather('ZIP:'+str(code), databaseid,begindate,enddate,mytoken,baseurl)
                fip_weather = fip_weather.append(newdf)
            except:
                continue
        if not fip_weather.empty:
            #average across the county
            precip = fip_weather[fip_weather['datatype']=='PRCP'][['date','value']].rename(columns={'value':'precipitation'})
            temp = fip_weather[fip_weather['datatype']=='TOBS'][['date','value']].rename(columns={'value':'temp'})
            w = pd.merge(precip, temp, how ='outer', on ='date') 
            avgs = pd.merge(w.groupby('date', as_index=False)['precipitation'].mean(),
                            w.groupby('date', as_index=False)['temp'].mean(),on='date')
            avgs['FIPS']=fip
            avgs = avgs.to_dict()

            #insert records into MongoDB
            a = [{key:value[index] for key,value in avgs.items()}
                 for index in range(max(map(len,avgs.values())))]
            records.insert_many(a)
            print(records.count())
        else:
            empty_fips.append(fip)
    
toc= time.perf_counter()
print(f"Run time was {toc - tic:0.4f} seconds")

9001
9003
9005
9007
9009
9011
9013
9015
12109


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere


5735
12111
5772
12119
5797
12121
5822
12127
5847
12129
5872
12131
5909
13013
5943
13015
5976
13021
6016
13029
6026
13031
6065
13033
6078
13039
6092
13045
6120
13047
6146
13051
6171
13057
6208
13059
6248
13063
6278
13067
6321
13073
6334
13077
6365
13085
6388
13089
6414
13095
6456
13097
6485
13111
6513
13113
6548
13115
6573
13117
6609
13119
6650
13121
6691
13127
6716
13135
6741
13139
6784
13143
6809
13145
6851
13151
6887
13153
6917
13179
6949
13185
6974
13207
7015
13215
7029
13217
7063
13223
7095
13241
7123
13245
7150
13247
7184
13263
7209
13291
7236
13295
7263
13311
7288
13313
7317
15001
7350
15003
7398
15007
7417
15009
7460
16001
7494
16005
7521
16015
7534
16017
7567
16019
7592
16027
7626
16043
7665
16051
7697
16055
7726
16069
7768
16083
7794
16085
7807
17019
7851
17031
7910
17043
7967
17077
8001
17087
8026
17089
8067
17093
8094
17097
8151
17099
8201
17119
8226
17111
8263
17113
8317
17143
8344
17161
8361
17167
8395
17163
8423
17179
8464
17197
8508
17199
8533
17201
8587
18003
8638
18005

#### Practice- only with DC zip codes:

In [ ]:
dc_zips = fips_to_zips['District of Columbia']
len(dc_zips)

In [ ]:
baseurl = 'https://www.ncdc.noaa.gov/cdo-web/api/v2/data?'
mytoken = 'NVPWXeGxislWKqThiizoffNjnPtUtAcT'
databaseid = 'GHCND'
begindate = '2020-03-01'
enddate = '2020-05-01'
dc_weather = pd.DataFrame()
for code in dc_zips[0:100]:
    print(code)
    try:
        newdf = get_weather('ZIP:'+str(code), databaseid,begindate,enddate,mytoken,baseurl)
        newdf.insert(0,'ZIP',code)
        dc_weather = dc_weather.append(newdf)
    except:
        continue

In [ ]:
precip = dc_weather[dc_weather['datatype']=='PRCP'][['ZIP','date','value']].rename(columns={'value':'precipitation'})
temp = dc_weather[dc_weather['datatype']=='TOBS'][['date','value']].rename(columns={'value':'temp'})
w = pd.merge(precip, temp, how ='outer', on ='date') 
avgs = pd.merge(w.groupby('date', as_index=False)['precipitation'].mean(),
                w.groupby('date', as_index=False)['temp'].mean(), on='date')
avgs ['County']='DC'
avgs.to_dict()

In [ ]:
ld = [{key:value[index] for key,value in a.items()}
         for index in range(max(map(len,a.values())))]
ld

#### Check whether weather from one ZIP is different across a county

In [ ]:
import scipy.stats as stats

In [ ]:
stats.f_oneway(weather['precipitation'][weather['ZIP'] == weather['ZIP'].unique()[0]],
               weather['precipitation'][weather['ZIP'] == weather['ZIP'].unique()[1]],
               weather['precipitation'][weather['ZIP'] == weather['ZIP'].unique()[2]])

In [ ]:
stats.f_oneway(weather['temp'][weather['ZIP'] == weather['ZIP'].unique()[0]].dropna(), 
             weather['temp'][weather['ZIP'] == weather['ZIP'].unique()[1]].dropna(),
             weather['temp'][weather['ZIP'] == weather['ZIP'].unique()[2]].dropna())

### Getting weather data using the noaa package from github:

In [ ]:
n = noaa.NOAA()

In [ ]:
forecast = n.get_observations(20001, 'US',start='2020-04-28',end='2020-04-30')
f = pd.DataFrame(forecast)
items = ['timestamp','textDescription','temperature','precipitationLast6Hours','relativeHumidity','cloudLayers']
f = f[items]
f.insert(0,'ZIP',zip_code) #add zip code 

In [ ]:
dc_weather = pd.DataFrame(columns = ['ZIP','timestamp','textDescription','temperature','precipitationLast6Hours','relativeHumidity','cloudLayers'])
dczips = [20001,20004, 20005, 20036, 20009, 20007, 20010, 20008, 20016]
for code in dczips:
    forecast = n.get_observations(code, 'US',start='2020-04-28',end='2020-04-30')
    for obs in forecast:
        time = obs['timestamp']
        desc = obs['textDescription']
        temp = obs['temperature']['value']
        precip = obs['precipitationLast6Hours']['value']
        hum = obs['relativeHumidity']['value']
        cloud = obs['cloudLayers'][0]['base']['value']
    
        row = [code,time,desc,temp,precip,hum,cloud]
        weather.loc[len(weather)] = row

In [ ]:
weather.to_csv('/Users/samismalling/Documents/mobility-report-data-extractor-master/CovidMobile/data/DC_weather_04_28-04_30.csv')

In [ ]:
weather = pd.DataFrame()
bad_zips = []
for zip_code in fips_to_county['ZIP'][0:200]:
    if zip_code not in bad_zips:
        print(zip_code)
        try:
            forecast = n.get_forecasts(zip_code, 'US',hourly = False)
            f = pd.DataFrame(forecast)
            f = f[['startTime','endTime','temperature','windSpeed','windDirection','shortForecast']]
            f.insert(0,'ZIP',zip_code) #add zip code 
            weather = pd.concat([weather, f], ignore_index = True)
        except:
            bad_zips.append(zip_code)
            continue